In [3]:
import json, yaml
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/Users/jnk789/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(torch.backends.mps.is_available())

True


## Load Models

In [5]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [6]:
model_name = "Qwen/Qwen2-7B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:57<00:00, 14.44s/it]


RuntimeError: MPS backend out of memory (MPS allocated: 20.34 GB, other allocations: 464.00 KB, max allowed: 20.40 GB). Tried to allocate 259.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

## Load Data

In [10]:
with open('../data/process_data.json', 'r') as f:
    data = json.load(f)

In [21]:
print(f"Number of data points: {len(data)}")
print(f"Keys: {data.keys()}")

print(f"Type of Data in the first key: {type(data['1000'])}")
print(f"Keys in the first data point: {data['1000'].keys()}")

print(data['1000']['1973.0']['description'])
print(data['1000']['1973.0']['label'])

Number of data points: 625
Keys: dict_keys(['1000', '1010', '1019', '1020', '1026', '1039', '1040', '1045', '1070', '1072', '1074', '1075', '1078', '1093', '1097', '1104', '1113', '1114', '1118', '1120', '1135', '1138', '1150', '1153', '1156', '1159', '1177', '1178', '1180', '1181', '1194', '1196', '1213', '1215', '1218', '1219', '1220', '1221', '1223', '1230', '1232', '1234', '1235', '1236', '1252', '1253', '1254', '1274', '1277', '1278', '1279', '1281', '1283', '1284', '1290', '1291', '1297', '1300', '1304', '1305', '1306', '1308', '1318', '1320', '1322', '1325', '1341', '1349', '1356', '1362', '1365', '1366', '1379', '1380', '1391', '1392', '1394', '1397', '1402', '1408', '1409', '1411', '1414', '1420', '1425', '1427', '1430', '1434', '1440', '1447', '1456', '1457', '1458', '1465', '1478', '1481', '1487', '1489', '1496', '1498', '1501', '1506', '1510', '1512', '1526', '1528', '1534', '1537', '1542', '1544', '1547', '1552', '1555', '1556', '1567', '1571', '1573', '1581', '1587', '159

In [27]:
# Construct the prompt
with open("../config/prompts.yaml", "r") as f:
    prompts = yaml.safe_load(f)
    
print(prompts['prompt_1'][0].format(balance_income_sheet=data['1000']['1973.0']['description']))

Based on the following financial statements, assess whether EPS will increase or decrease in the next year:

Below is the Balance Sheet:
------------------------------------------------------------------------------------------
| Account Items                                                  | 1971 | 1972 |
------------------------------------------------------------------------------------------
| Total Asset                                                  | 29.330 | 19.907 |
| Current Assets                                               | 19.688 | 11.326 |
| Current Liabilities                                          | 8.507 | 3.990 |
| Cash and Short-Term Investments                              | 2.557 | 2.027 |
| Receivables                                                  | 8.398 | 4.186 |
| Inventories                                                  | 8.187 | 4.743 |
| Other Current Assets                                         | 0.546 | 0.370 |
| Property, Plant, and Equipm

In [28]:
prompt_input = prompts['prompt_1'][0].format(balance_income_sheet=data['1000']['1973.0']['description'])

## Inference

In [29]:
# prepare the model input
# prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt_input}
]

# Format prompt using chat template
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nBased on the following financial statements, assess whether EPS will increase or decrease in the next year:\n\nBelow is the Balance Sheet:\n------------------------------------------------------------------------------------------\n| Account Items                                                  | 1971 | 1972 |\n------------------------------------------------------------------------------------------\n| Total Asset                                                  | 29.330 | 19.907 |\n| Current Assets                                               | 19.688 | 11.326 |\n| Current Liabilities                                          | 8.507 | 3.990 |\n| Cash and Short-Term Investments                              | 2.557 | 2.027 |\n| Receivables                                                  | 8.398 | 4.186 |\n| Inventories                                                  | 8.187 | 4.743 |\n| Other Current Ass

In [32]:
# Generate response
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
model_inputs

{'input_ids': tensor([[151644,   8948,    198,  ..., 151644,  77091,    198]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='mps:0')}

In [ ]:
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)

In [ ]:
# Decode and print output
response = tokenizer.batch_decode(
    generated_ids[:, model_inputs.input_ids.shape[1]:], 
    skip_special_tokens=True
)[0]


print(response)